In [2]:
pip install pandas_ta stable_baselines3 shimmy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of shimmy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 33.6 MB/s eta 0:00:00
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=d7352921a0fe3becb8ed8591f6e0ed4cc4f21df84e43e7589fd3463ea2eb560f
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
import pandas as pd
import yfinance as yf

# function to get stock from yfinance
def get_stock_data_with_indicators(stock_symbols):
    stock_data = pd.DataFrame()

    # --- loop to get HLOC and compute indicator ema10 and ema60 + RSI
    for i, symbol in enumerate(stock_symbols):

        # download data
        data = yf.download(symbol, period="5y", interval="1wk")

        # HLOC
        stock_data[f'low_{i}'] = data['Low']
        stock_data[f'close_{i}'] = data['Close']
        stock_data[f'high_{i}'] = data['High']
        stock_data[f'open_{i}'] = data['Open']
        stock_data[f'volume_{i}'] = data['Volume']

        # ema10 and ema60
        stock_data[f'ema10_{i}'] = data['Close'].ewm(span=10, adjust=False).mean()
        stock_data[f'ema60_{i}'] = data['Close'].ewm(span=60, adjust=False).mean()

        # RSI
        delta = data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        stock_data[f'rsi_{i}'] = 100 - (100 / (1 + rs))

    # add week infos
    stock_data['week'] = data.index.isocalendar().week  # Add week number info

    # important: handle NaN values
    stock_data.fillna(method='ffill', inplace=True)  # Forward fill NaN values
    stock_data.fillna(method='bfill', inplace=True)  # Backward fill if any initial NaN values remain

    return stock_data.reset_index(drop=True)

# stocks to download
stock_symbols = [
    "EURUSD=X"
]

# create the stock_data for agent
stock_data = get_stock_data_with_indicators(stock_symbols)

[*********************100%***********************]  1 of 1 completed
<ipython-input-3-ed6af78a13ad>:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data.fillna(method='ffill', inplace=True)  # Forward fill NaN values
<ipython-input-3-ed6af78a13ad>:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data.fillna(method='bfill', inplace=True)  # Backward fill if any initial NaN values remain


In [4]:
import gym
import numpy as np
import pandas as pd
import yfinance as yf
from stable_baselines3 import PPO
from gym import spaces

class StockEnv(gym.Env):
    def __init__(self, stock_data, initial_equity=100000, max_bets=10, history_weeks=4):
        super(StockEnv, self).__init__()
        self.stock_data = stock_data.reset_index(drop=True)  # index reset
        self.current_week = history_weeks  # shift to have historic data
        self.max_bets = max_bets
        self.max_weeks = len(stock_data)
        self.history_weeks = history_weeks  # past data

        # equity initialisation
        self.equity = initial_equity

        # get num of stocks (important: implied 8 features for stocks)
        self.num_stocks = (len(stock_data.columns) - 1) // 8

        # action space is binary, bet or not
        self.action_space = spaces.MultiBinary(self.num_stocks)

        # observation space: past week data (8 features per stock for history_weeks) + equity
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=((self.num_stocks * 8 * self.history_weeks) + 1,),
            dtype=np.float32
        )

    def reset(self):
        # reset environment to initial space
        self.current_week = self.history_weeks
        self.equity = 100000  # reset equity
        return self._get_observation()  # get current obs

    def step(self, action):
        # ensure action is valid (binary for each stock)
        action = np.clip(action, 0, 1)

        # get current week's data
        current_data = self.stock_data.iloc[self.current_week]
        next_data = self.stock_data.iloc[self.current_week + 1] if self.current_week + 1 < self.max_weeks else None

        reward = 0
        done = False

        # ---> loop to check if current close price is above EMA10 for each stock (added condition)
        for i in range(self.num_stocks):
            close_this_week = current_data[f'close_{i}']
            ema10_this_week = current_data[f'ema10_{i}']

            if action[i] == 1:  # agent has bet on this stock
                # allow trading only if close price is above EMA10, instead of penalizing
                if close_this_week > ema10_this_week:
                    low_next_week = next_data[f'low_{i}'] if next_data is not None else None
                    close_next_week = next_data[f'close_{i}'] if next_data is not None else None

                    if low_next_week is not None:
                        # check stop-loss condition
                        if low_next_week > current_data[f'low_{i}']:
                            reward -= 1000  # loss of 1%
                            self.equity -= 1000  # deducted from equity
                        else:
                            # avoid division by zero by checking close_this_week - low_this_week
                            denominator = close_this_week - current_data[f'low_{i}']
                            if denominator == 0:
                                ratio = 0  # assign a default value
                            else:
                                ratio = (close_next_week - close_this_week) / denominator

                            gain = ratio * 1000
                            reward += gain
                            self.equity += gain  # add gain to equity if win

        # if agent picks no stocks, apply a small penality (push the agent to explore)
        if sum(action) == 0:
            reward -= 10  # low penality

        # move to the next week
        self.current_week += 1

        # check if the equity is below zero (end of the game)
        if self.equity <= 0:
            done = True
            reward -= 100000  # extra penalty for going bankrupt

        if self.current_week >= self.max_weeks - 1:
            done = True

        # get next observation (for next week)
        next_observation = self._get_observation()

        return next_observation, reward, done, {}

    def _get_observation(self):
        # get the last `history_weeks` of data for each stock and current equity
        history_data = self.stock_data.iloc[self.current_week - self.history_weeks: self.current_week]
        obs = []

        # ---> loop to get the features for each stock
        for i in range(self.history_weeks):
            week_data = history_data.iloc[i].values[:-1]  # exclude 'week' column, include all other features
            obs.extend(week_data)

        # add current equity to the observation
        obs.append(self.equity)

        return np.array(obs, dtype=np.float32)

    def render(self, mode='human'):
        print(f"Week: {self.current_week}, Equity: {self.equity}")

# ----- Agents

# initialize environment
env = StockEnv(stock_data)

# train using PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# test the agent
obs = env.reset()
for _ in range(100):
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()  # render current week and equity
    if done:
        break


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 257      |
|    ep_rew_mean     | -5.7e+04 |
| time/              |          |
|    fps             | 502      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 257          |
|    ep_rew_mean          | -5.93e+04    |
| time/                   |              |
|    fps                  | 393          |
|    iterations           | 2            |
|    time_elapsed         | 10           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0017882187 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.693       |
|    explained_variance   | 0            |
|    learning_r